<a href="https://colab.research.google.com/github/davidvlaminck/AWVGeneric/blob/master/UseCases/SearchAssetsWithParent/OTL_assets_met_naampad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pas eerst onderstaande parameter aan en voer het script uit. Een cookie vervalt binnen de 24 uur. Je kan [hier](https://sites.google.com/mow.vlaanderen.be/em-infra/afspraken-conventies/awv-cookie-ophalen) lezen hoe je de AWV ACM cookie kan ophalen.

In [ ]:
awv_acm_cookie = '6043f21b7cfe44568b87d2cacb120063'

Voer onderstaande script uit om de GitHub repo te clonen.

In [ ]:
import os, shutil
if os.path.exists('AWVGeneric'):
  shutil.rmtree('AWVGeneric')
! git clone https://github.com/davidvlaminck/AWVGeneric

Cloning into 'AWVGeneric'...
remote: Enumerating objects: 180, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 180 (delta 107), reused 123 (delta 55), pack-reused 0 (from 0)
Receiving objects: 100% (180/180), 39.61 KiB | 3.60 MiB/s, done.
Resolving deltas: 100% (107/107), done.


Voer onderstaande script uit om de SN Gateway te query'en

In [ ]:
import sys
sys.path.insert(0, '/content/AWVGeneric')  # adds the Python files from the GitHub repo

from pathlib import Path
from prettytable import PrettyTable

from API.eminfra.eminfra_client import EMInfraClient
from API.EMInfraDomain import QueryDTO, PagingModeEnum, SelectionDTO, ExpressionDTO, TermDTO, OperatorEnum, \
    LogicalOpEnum, ExpansionsDTO, construct_naampad
from API.Enums import AuthType, Environment

eminfra_client = EMInfraClient(cookie=awv_acm_cookie, auth_type=AuthType.COOKIE, env=Environment.PRD)  # change environment here

asset_types = list(eminfra_client.get_all_otl_assettypes())
print(f'aantal OTL types: {len(asset_types)}')

type_term = TermDTO(property='type', operator=OperatorEnum.EQ, value='a7eadedf-b5cf-491b-8b89-ccced9a37004')
query_dto = QueryDTO(size=100, from_=0, pagingMode=PagingModeEnum.OFFSET,
                      expansions=ExpansionsDTO(fields=['parent']),
                      selection=SelectionDTO(
                          expressions=[ExpressionDTO(
                              terms=[type_term,
                                  TermDTO(property='actief', operator=OperatorEnum.EQ,
                                          value=True, logicalOp=LogicalOpEnum.AND),
                                  TermDTO(property='beheerobject', operator=OperatorEnum.EQ,
                                          value=None, logicalOp=LogicalOpEnum.AND, negate=True),
                                  TermDTO(property='naamPad', operator=OperatorEnum.STARTS_WITH,
                                          value='DA-', logicalOp=LogicalOpEnum.AND, negate=True),
                                  TermDTO(property='naamPad', operator=OperatorEnum.STARTS_WITH,
                                          value='OTN.', logicalOp=LogicalOpEnum.AND, negate=True)])]))
headers = ['uuid', 'type', 'naampad', 'em_infra_link']
rows = []
for otl_asset_type in asset_types:
    print(f'querying type {otl_asset_type.korteUri}')
    type_term.value = otl_asset_type.uuid
    query_dto.from_ = 0
    rows.extend([asset.uuid, otl_asset_type.korteUri, construct_naampad(asset),
                  f'https://apps.mow.vlaanderen.be/eminfra/assets/{asset.uuid}']
                for asset in eminfra_client.search_assets(query_dto))
table = PrettyTable(headers)
table.add_rows(rows)

with open('otl_asset_with_naampad.csv', 'w', newline='') as f_output:
    f_output.write(table.get_csv_string())


aantal OTL types: 745
querying type onderdeel#WazeBeacon
querying type onderdeel#IVSB
querying type onderdeel#Bijlage
querying type onderdeel#DigitaleHoppinzuil
querying type onderdeel#Stuwconstructie
querying type onderdeel#Stortsteenpenetratie
querying type installatie#Sluishoofd
querying type onderdeel#Fendering
querying type installatie#Vloernis
querying type installatie#Hameistijl
querying type installatie#Balansarm
querying type installatie#Aanslagbalk
querying type installatie#Stuwhoofd
querying type onderdeel#Toegangsvoorziening
querying type onderdeel#Asfaltmat
querying type onderdeel#Ballastcompartiment
querying type installatie#Drempel
querying type onderdeel#Haalkom
querying type implementatieelement#Bijlage
querying type onderdeel#Betonmatras
querying type installatie#Bodembescherming
querying type onderdeel#Werkvloer
querying type installatie#Hanger
querying type onderdeel#Wiepenmat
querying type onderdeel#Draineerlaag
querying type onderdeel#Centreerinrichting
querying t